# Language Data
by Elmas Kat 2828399

Part 1

In [9]:
import pandas as pd
from sklearn.metrics import cohen_kappa_score, confusion_matrix

# Selinay
selinay_df = pd.read_csv(r"C:\Users\KatElmas\OneDrive - van der Meij college\bureaublad\selinay_annotations.csv")
kappa_selinay = cohen_kappa_score(selinay_df["elmas"], selinay_df["carla"])
print("Cohen's kappa (Selinay, Elmas vs Carla):", kappa_selinay)

# Gultanem
gultanem_df = pd.read_csv(r"C:\Users\KatElmas\OneDrive - van der Meij college\bureaublad\gultanem_annotations.csv")
kappa_gultanem = cohen_kappa_score(gultanem_df["elmas"], gultanem_df["carla"])
print("Cohen's kappa (Gultanem, Elmas vs Carla):", kappa_gultanem)


Cohen's kappa (Selinay, Elmas vs Carla): 0.4383187772925764
Cohen's kappa (Gultanem, Elmas vs Carla): 0.48039215686274517


In [10]:
from sklearn.metrics import confusion_matrix

# Confusion matrix Selinay 
cm_selinay = confusion_matrix(
    selinay_df["carla"],    
    selinay_df["elmas"]     
)

print("Confusion matrix Selinay (rows = Carla, cols = Elmas):")
print(cm_selinay)

Confusion matrix Selinay (rows = Carla, cols = Elmas):
[[ 0  0  0  0  1  0  0  0  0  0  0]
 [ 0  1  0  2  0  0  0  0  0  0  4]
 [ 0  1  0  1  0  0  0  0  0  1  1]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  1  1]
 [ 0  0  0  0  0  3  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  7  0  0  0]
 [ 0  0  0  0  0  0  0  1  0  0  0]
 [ 0  0  0  0  2  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  1  1  0 16]]


In [11]:
# Confusion matrix Gultanem 
cm_gultanem = confusion_matrix(
    gultanem_df["carla"],   
    gultanem_df["elmas"]    
)

print("\nConfusion matrix Gultanem (rows = Carla, cols = Elmas):")
print(cm_gultanem)


Confusion matrix Gultanem (rows = Carla, cols = Elmas):
[[ 3  0  2  0  0  0  2  0  0  0  1]
 [ 3  0  2  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  2  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  2  0  1]
 [ 0  0  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 10  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  0  0  0]
 [ 0  0  0  1  0  0  0  0  1  0  1]
 [ 0  0  0  0  0  0  0  1  0  0  0]
 [ 0  0  0  0  0  0  1  1  0  0 16]]


In [12]:
# % agreement Selinay 
agree_selinay = (selinay_df["carla"] == selinay_df["elmas"]).mean()
print("Percentage agreement Selinay vs Carla:", round(agree_selinay * 100, 1), "%")

# % agreement Gultanem 
agree_gultanem = (gultanem_df["carla"] == gultanem_df["elmas"]).mean()
print("Percentage agreement Gultanem vs Carla:", round(agree_gultanem * 100, 1), "%")

Percentage agreement Selinay vs Carla: 57.1 %
Percentage agreement Gultanem vs Carla: 58.5 %


Part 3

In [19]:
import pandas as pd
from langchain_ollama import ChatOllama
import json
from datetime import datetime
from sklearn.metrics import cohen_kappa_score, confusion_matrix

ELMAS_LABELS = [
    "STATEMENT",
    "ACKNOWLEDGEMENT",
    "CLARIFICATION",
    "QUESTION",
    "OTHER",
    "REPAIR",
    "EMOTIVE",
    "EXPRESSIVE",
    "BACKCHANNEL"
]

In [29]:
# HLT course code

def safe_parse_json(text):
    """Return JSON if possible, otherwise OTHER."""
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        return {"label": "OTHER", "raw_output": text}


class LLMAnnotator:
    def __init__(self, model="gpt-oss:20b", labels=None, max_context=3, temperature=0.2):
        if labels is None:
            labels = ELMAS_LABELS

        self._labels = labels
        self._max_context = max_context
        self._history = []

        # Ollama client
        self._client = ChatOllama(
            model=model,
            think=False,
            temperature=temperature,
        )

        self._build_system_prompt()

    def _build_system_prompt(self):
        self._instruct = [
            {"role": "system",
             "content": "You are a dialogue act annotator."},

            {"role": "system",
             "content": "Always label ONLY the LAST utterance."},

            {"role": "system",
             "content": f"Use only these labels: {self._labels}."},

            {"role": "system",
             "content": (
                 "Always output EXACTLY this format:\n"
                 "{\"label\": \"STATEMENT\"}\n"
                 "No extra text. No explanation."
             )},
        ]

    # annotate 1 utterance 
    def annotate_utterance(self, context, target):
        context_text = ""
        if context:
            context_text += "Previous:\n"
            for u in context:
                context_text += f"- {u}\n"

        context_text += f"Last:\n- {target}"

        prompt = self._instruct + [
            {"role": "user", "content": context_text}
        ]

        response = self._client.invoke(prompt)
        raw = response.content.strip()

        if "</think>" in raw:
            raw = raw.split("</think>")[-1].strip()

        return safe_parse_json(raw)

    # annotate complete conversation 
    def annotate_conversation(self, utterances):
        labels = []
        context = []

        print(f"Annotating {len(utterances)} utterances...\n")
        start = datetime.now()

        for i, utt in enumerate(utterances):
            output = self.annotate_utterance(context, utt)
            label = output.get("label", "OTHER")
            labels.append(label)

            context.append(utt)
            if len(context) > self._max_context:
                context = context[1:]

            if (i + 1) % 10 == 0:
                print(f"{i+1}/{len(utterances)} done...")

        duration = datetime.now() - start
        print(f"\nCompleted in {duration.seconds} sec")
        return labels

    
def annotate_corpus(self, utterances):
        return self.annotate_conversation(utterances)


In [21]:
import pandas as pd

iaa_df = pd.read_csv(
    r"C:\Users\KatElmas\OneDrive - van der Meij college\bureaublad\Elmas_combined_annotations.csv"
)

print("Number of rows:", len(iaa_df))
iaa_df.head()

Number of rows: 102


conversation  speaker                                          utterance  \
0      Selinay  Selinay  en toen stonden er allemaal politie mochten we...   
1      Selinay  Selinay                                    nee maar ik [ ]   
2      Selinay  Selinay     ohja wij dachten dat king charles erin zat [ ]   
3      Selinay  Selinay                                         ((laughs))   
4      Selinay  Selinay                           maar het was echt druk =   

       elmas_label    carla_label  
0        STATEMENT      STATEMENT  
1    CLARIFICATION         REPAIR  
2  ACKNOWLEDGEMENT         ANSWER  
3            OTHER          OTHER  
4        STATEMENT  CLARIFICATION

In [22]:
dialogue_act_labels = ELMAS_LABELS

llm_annotator = LLMAnnotator(
    model="gpt-oss:20b",        
    labels=dialogue_act_labels,
    max_context=102,            
    temperature=0.2
)

# Extract the utterances from dataframe
utterances = iaa_df["utterance"].tolist()   

# Run the LLM annotation
llm_labels = llm_annotator.annotate_conversation(utterances)

iaa_df["llm_label"] = llm_labels

# Preview the result
iaa_df[["conversation", "speaker", "utterance", "elmas_label", "carla_label", "llm_label"]].head()

Annotating 102 utterances...

10/102 done...
20/102 done...
30/102 done...
40/102 done...
50/102 done...
60/102 done...
70/102 done...
80/102 done...
90/102 done...
100/102 done...

Completed in 3485 sec


conversation  speaker                                          utterance  \
0      Selinay  Selinay  en toen stonden er allemaal politie mochten we...   
1      Selinay  Selinay                                    nee maar ik [ ]   
2      Selinay  Selinay     ohja wij dachten dat king charles erin zat [ ]   
3      Selinay  Selinay                                         ((laughs))   
4      Selinay  Selinay                           maar het was echt druk =   

       elmas_label    carla_label  llm_label  
0        STATEMENT      STATEMENT  STATEMENT  
1    CLARIFICATION         REPAIR  STATEMENT  
2  ACKNOWLEDGEMENT         ANSWER  STATEMENT  
3            OTHER          OTHER    EMOTIVE  
4        STATEMENT  CLARIFICATION  STATEMENT

In [23]:
from sklearn.metrics import cohen_kappa_score, confusion_matrix

# Kappa: Elmas vs LLM
kappa_elmas_llm = cohen_kappa_score(iaa_df["elmas_label"], iaa_df["llm_label"])
print("Cohen's kappa (Elmas vs LLM):", kappa_elmas_llm)

# Kappa: Carla vs LLM
kappa_carla_llm = cohen_kappa_score(iaa_df["carla_label"], iaa_df["llm_label"])
print("Cohen's kappa (Carla vs LLM):", kappa_carla_llm)

# Confusion matrices
label_order = ELMAS_LABELS

cm_elmas = confusion_matrix(iaa_df["elmas_label"], iaa_df["llm_label"], labels=label_order)
cm_carla = confusion_matrix(iaa_df["carla_label"], iaa_df["llm_label"], labels=label_order)

print("\nConfusion matrix (Elmas vs LLM)")
print(label_order)
print(cm_elmas)

print("\nConfusion matrix (Carla vs LLM)")
print(label_order)
print(cm_carla)

# Percentage agreement
agreement_elmas = (iaa_df["elmas_label"] == iaa_df["llm_label"]).mean()
agreement_carla = (iaa_df["carla_label"] == iaa_df["llm_label"]).mean()

print("\nPercentage agreement (Elmas vs LLM):", round(agreement_elmas * 100, 1), "%")
print("Percentage agreement (Carla vs LLM):", round(agreement_carla * 100, 1), "%")


Cohen's kappa (Elmas vs LLM): 0.2806184728061847
Cohen's kappa (Carla vs LLM): 0.22338366240805263

Confusion matrix (Elmas vs LLM)
['STATEMENT', 'ACKNOWLEDGEMENT', 'CLARIFICATION', 'QUESTION', 'OTHER', 'REPAIR', 'EMOTIVE', 'EXPRESSIVE', 'BACKCHANNEL']
[[41  1  0  1  0  0  0  0  0]
 [ 6  1  0  0  0  0  0  0  2]
 [ 5  0  0  0  0  0  0  0  0]
 [ 1  0  0  3  0  0  0  0  0]
 [ 4  0  0  1  3  0  8  7  1]
 [ 4  1  0  0  0  0  0  0  0]
 [ 2  0  0  0  0  0  1  1  0]
 [ 1  0  0  0  0  0  0  0  0]
 [ 1  5  0  0  0  0  0  0  1]]

Confusion matrix (Carla vs LLM)
['STATEMENT', 'ACKNOWLEDGEMENT', 'CLARIFICATION', 'QUESTION', 'OTHER', 'REPAIR', 'EMOTIVE', 'EXPRESSIVE', 'BACKCHANNEL']
[[34  0  0  2  0  0  1  0  0]
 [ 6  4  0  0  1  0  0  0  4]
 [ 6  0  0  0  0  0  0  0  0]
 [ 0  0  0  2  0  0  0  0  0]
 [ 2  0  0  0  2  0  6  7  0]
 [ 5  0  0  0  0  0  0  0  0]
 [ 3  0  0  0  0  0  1  1  0]
 [ 2  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  1  0  0]]

Percentage agreement (Elmas vs LLM): 49.0 %
Percen

utterances on finding the best prompt

In [ ]:
ELMAS_LABELS = [
    "STATEMENT",
    "ACKNOWLEDGEMENT",
    "CLARIFICATION",
    "QUESTION",
    "OTHER",
    "REPAIR",
    "EMOTIVE",
    "EXPRESSIVE",
    "BACKCHANNEL"
]

In [24]:
utterances = [
    " *laughing* ja we dachten wat gebeurt er allemaal",
    "nee maar ik…",
    "nee nee nee wacht",
    "eerste keer dat we daar zo langs liepen toen",
    "waren toch van die uh",
    "uh limousine en toen dachten we wie zit daar in enzo",
    "ohja wij dachten dat king charles erin zat",
    "wist je nog"
]

In [25]:
import subprocess

def annotate_with_ollama(text):
    prompt = f"""
You are an expert dialogue act annotator.

Use EXACTLY ONE of these labels:
STATEMENT, ACKNOWLEDGEMENT, CLARIFICATION, QUESTION, OTHER,
REPAIR, EMOTIVE, EXPRESSIVE, BACKCHANNEL.

Give ONLY the label as answer.
No explanation, no JSON, no extra text. 
Just one word: the label.

Utterance: "{text}"
"""

    result = subprocess.run(
        ["ollama", "run", "gpt-oss:20b"],
        input=prompt.encode("utf-8"),
        stdout=subprocess.PIPE
    )

    output = result.stdout.decode("utf-8").strip()

    label = output.split()[-1].upper().strip('",.:')
    return label

In [26]:
llm_labels = []

for utt in utterances:
    label = annotate_with_ollama(utt)   
    llm_labels.append(label)
    print(utt, "→", label)

 *laughing* ja we dachten wat gebeurt er allemaal → STATEMENT
nee maar ik… → STATEMENT
nee nee nee wacht → EXPRESSIVE
eerste keer dat we daar zo langs liepen toen → STATEMENT
waren toch van die uh → STATEMENT
uh limousine en toen dachten we wie zit daar in enzo → STATEMENT
ohja wij dachten dat king charles erin zat → STATEMENT
wist je nog → QUESTION
